# Bioinformatics Analysis Protocol
## Breakdown:
### 1) Pre-processing Sequences
### 2) MSA analysis
### 3) Data Analysis

## 1) Pre-processing Sequences
### Goals: Take a large body of sequences (of varying lengths, qualities, sources, etc) and parse this dataset down into sequences of interest.
#### Specifically, I am interesting in looking at sequences related to the _helicase domain_ of the _flavivirus_ NS3 protein. 
Steps:

    1) Python script. Remove sequences that are not associated with the desired protein. Chuck a sequence if:
        
        a) Collect the segment of each sequence that is associated with the helicase domain. To do this, I use a regular expression to find sequence segments of the protein that are known to be strongly conserved. In this case, the regular expression is used to find Motifs I and II (Walker A and B), which are roughly separated by 90 residues in the flavivirus NS3h sequences. _user-defined variables_: regular_expression, average_expression_length, expression_length_deviation, number_residues_pre_expression

        b) the sequence is too short to be a sequence of interest; _user-defined variables_: average_sequence_length, sequence_length_deviation
        
        c) the sequence is poorly resolved (number of 'X', 'B', or 'Z' residues is beyond some cutoff); _user-defined variables_: prob_XBZ
            
        
    2) Hierarchical CD-HIT protocol. Removes identical sequences (every remaining sequence has equal weight); Also used to remove sequence outliers that made it through the python script. 
        https://github.com/weizhongli/cdhit/wiki/3.-User's-Guide#CDHIT

#### Step 1

In [ ]:
import re
from Bio import SeqIO

original_sequence_file = 'all_sources_NS3_sequences.fasta'
output_file_name_root = 'cleaned_up_a.fasta'
regular_expression = 'G[AS]GK(.+?)DE[AC]H' # regular expression formatting to find motifs I and II, naive about sequences between the given expected strings. 
average_expression_length = 90
expression_length_deviation = 25
number_residues_pre_expression = 20
average_sequence_length = 450
sequence_length_deviation = 30
prob_XBZ = 0.25 # cutoff for probability of poorly resolved residues in a sequence

sequences = list(SeqIO.parse(original_sequence_file,'fasta'))
nSeqs = len(sequences)
nSeqs_range = list(range(nSeqs))
minimum_sequence_length = average_sequence_length - sequence_length_deviation
maximum_sequence_length = average_sequence_length + sequence_length_deviation
minimum_expression_length = average_sequence_length - expression_length_deviation
maximum_expression_length = average_sequence_length + expression_length_deviation

sequence_dictionary = {}
with open(output_file_name_root+'.summary.txt','w') as W:
    for i in nSeqs_range:     # used to loop over all sequences in original_sequence_file 
        sequence = str(sequences[i].seq)
        last_found = -1
        found = []
        while True:     # finding all instances of the regular expression within the sequence; 
            try:
                search_string = re.search(regular_expression,sequence[last_found+1:]).group(0)     # finding first instance of regex
                zeroth_index_search_string = sequence.find(search_string)     # getting regex zeroth-index in sequence
                zeroth_index_sequence = zeroth_index_search_string - number_residues_pre_expression     # actually interested in including residues pre-Motif I, so finding that zeroth-index
                if zeroth_index_sequence < 0:     # if regex is found early in sequence, ignore the pre_expression residues zeroth_index, just take the whole sequence
                    found.append([zeroth_index_search_string,search_string,len(sequence[:]),sequence[:]])
                else:     # grab the sequence associated with pre_expression zeroth-index
                    found.append([zeroth_index_search_string,search_string,len(sequence[zeroth_index_sequence:]),sequence[zeroth_index_sequence:]])
            except AttributeError:     # if regex isn't found, break from the while loop
                break
            last_found = sequence.find(search_string,last_found+1)     # update the last_found variable to find more instances of the regex in sequence[i]
        
        if last_found == -1:     # last_found is never updated if no regex is found, which indicates that important motifs are not present in the sequence.
            W.write('%-5d %s Regular expression input not found; removed this sequence from the population of sequences.\n'%(i,sequences[i].id))
            continue
        for j in found:     # Analyze every instance of the regex being found within the sequence
            if j[2] < minimum_sequence_length:     # compare length of sequence to min sequence length; if too short, remove sequence
                W.write('%-5d %s Regular expression input found but the remaining sequence is too short (length = %d) to be the full, desired sequence.\n'%(i,sequences[i].id,j[2]))
                continue
            elif len(j[1]) < minimum_expression_length or len(j[1]) > maximum_expression_length:     # compare length of sequence of the regex; if of unexpected length, remove sequence
                W.Write('%-5d %s Regular expression input found but length of the search string is unexpected (length = %d) (expected length w/in %d to %d residues)'%(i,sequences[i].id,len(j[1]),minimum_expression_length,maximum_expression_length))
                continue
            elif j[2] > maximum_sequence_length:     # compare length of sequence to max sequence length; only grab the first avg_sequence_length residues and keep sequence
                sequence = j[3][:avg_sequence_length+1]
                W.write('%-5d %s Regular expression input found but length of sequence is longer than expected; truncating (post-search_string) segment assuming the sequence should be approximately the average length.'%(i,sequences[i].id))
            else:
                sequence = j[3]
                W.write('%-5d %s Regular expression input found; sequence length is within expected range. ' %(i,sequences[i].id))
            
            if float(sequence.count('X') + sequence.count('B') + sequence.count('Z'))/len(sequence) <= prob_XBZ:     # test if prob of a poorly resolved AA symbol is greater than prob_XBZ; if <=, then keep sequence 
                if sequence not in sequence_dictionary:     # test if sequence hasn't already been ID'ed; if it hasn't, create a new dictionary key and value
                    sequence_dictionary[sequence] = sequences[i].id
                    W.write('This sequence string is the first instance of such a sequence; Adding sequence to sequence dictionary.\n')
                else:     # if it has, add the sequence ID to the end of the dictionary value (which is the sequence ID of previously-found sequences)
                    sequence_dictionary[sequence] += '___' + sequences[i].id
                    W.write('This sequence string has been seen before; appending sequence info to previous sequence_dictioanry value.\n')
            else:
                W.write('This sequence string is poor resolution. Removed from the population of sequences.\n')

with open(output_file_name_root+'.fasta','w') as output_file:     # output the fasta file associated with this pre-processing
    for sequence in sequence_dictionary:
        output_file.write('>'+sequence_dictionary[sequence]+'\n'+sequence+'\n')
        

#### Step 2
Remove degenerate sequences that were not caught during step 1 as well as removing non-flavivirus NS3h sequences by using the CD-HIT software. See https://github.com/weizhongli/cdhit/wiki/3.-User's-Guide#CDHIT for a user guide.

"CD-HIT clusters proteins into clusters that meet a user-defined similarity threshold, usually a sequence identity. Each cluster has one representative sequence. The input is a protein dataset in fasta format and the output are two files: a fasta file of representative sequences and a text file of list of clusters."

##### Clustering step 1: threshold value = 1.0

Removes degenerate (completely identical) sequences within the ensemble. This step is important because it normalizes the weight associated with every unique sequence; we aren't interested in the frequency that a specific sequence of NS3h is observed, just the population of sequences. This might be a source of human bias. 

##### Clustering step 2: threshold value = 0.4

Using a small threshold value separates sequences into families (e.g. hepacivirus, pestiviruses, and flavivirus sequences) as well as removes extreme outliers that aren't clustered into a family.

The CD-HIT software package comes with a perl script that will take the clustering results and prepare a fasta file for each cluster. This file can then be used, for example, in a multisequence alignment analysis for each cluster.

How to use: 

"perl /home/apps/cdhit/make_multi_seq.pl original_sequence_file clustering_analysis_output_file.clstr new_directory_name num_sequences_in_cluster"

where 

      original_sequence_file is the input fasta file to the cd-hit analysis

      clustering_analysis_output_file.clstr is the cd-hit cluster output file 
      
      new_directory_name is a directory to be created to hold the clusters' fasta files to be created
      
      num_sequence_in_cluster is an integer value that functions as a minimum number of sequences in a cluster to be analyzed further; this variable is used to remove extreme outliers from the weak clustering threshold

Initially, I am interested in looking at only flaviviral sequences (which is likely associated to cluster 1 and respective file named '1').

##### Clustering step 3: threshold value = 0.8

Cluster sequences one last time with a consistency cutoff of 0.8. Use the perl script to output well represented clusters. Combine the cluster fasta files into one to be analyzed in the MSA.

## 2) Multiple Sequence Alignment (MSA) Analysis
https://mafft.cbrc.jp/alignment/software/algorithms/algorithms.html
### Goals: Take processed sequences and calculate a high quality MSA. There are numerous methods within the MAFFT software available to do so. :. perform MSA using a subset of these methods and compare results.
#### Specifically, I am interested in obtaining the highest quality MSA possible. Unfortunately, I am unfamiliar with the MSA algorithms and scoring functions used within the software, so I will initially use brute force to obtain the desired MSA.
Steps:

    1) Run MAFFT software on the same sequence file using three different methods. Outputs will all be different. 
    
    2) Use the TRIMAL software to quantitatively decide which MSA result is best. 

#### Step 1
Progressive alignment with iterative refinement: FFT-NS-i

Local alignment using iterative refinement methods using WSP and consistency scores: L-INS-i

Global alignment using iterative refinement methods using WSP and consistency scores: G-INS-i

#### Step 2
Trimal software allows for quantitative comparison of MSA alignments. To do so, I need to create a txt file that has the locations of each msa file on new lines (compareset_input_file.txt).

Using this function, trimal compares the mafft methods to identify the highest quality alignment based on some (unknown to me) scoring function and outputs the respective MSA file with columns removed that have gaps in more than 20% of all sequences. The choice of 20% is a potential source of bias. 

## Data Analysis 
### Goals: Mine the MSA results to obtain novel insight into the biophysics of the NS3 helicase domain. Couple the insights gained from MD simulations with the bioinformatics results stemming from the MSA. 
#### Specifically, I am interested in using the MSA results to guide my analysis of MD simulations (e.g. choice of collective variables that are strongly conserved across all flaviviruses, allowing for their use in numerous bodies of simulations). 
Steps:

    1) Use a python script to calculate basic results from the MSA. Specifically, the consensus sequence, position frequency matrix, and mutual information. Interpret/couple these results with the MD simulations.
    
    2) Use the weblogo (http://weblogo.threeplusone.com/) web-based application to create a figure presenting the MSA results. 
    

Post-analysis python script usage: